<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# Automatically find the bad LLM responses in your LLM Evals with Cleanlab

This guide will walk you through the process of evaluating LLM responses captured in Phoenix with Cleanlab's Trustworthy Language Models (TLM).

TLM boosts the reliability of any LLM application by indicating when the model’s response is untrustworthy.

This guide requires a Cleanlab TLM API key. If you don't have one, you can sign up for a free trial [here](https://tlm.cleanlab.ai/).

## Install dependencies & Set environment variables

In [ ]:
%%bash
pip install -q "arize-phoenix>=4.29.0"
pip install -q 'httpx<0.28'
pip install -q openai cleanlab_tlm openinference-instrumentation-openai

In [ ]:
import json
import os
from getpass import getpass

import dotenv

dotenv.load_dotenv()

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
# Sign up for a free trial of Cleanlab TLM and get an API key [here](https://tlm.cleanlab.ai/).
if not (cleanlab_tlm_api_key := os.getenv("CLEANLAB_TLM_API_KEY")):
    cleanlab_tlm_api_key = getpass("🔑 Enter your Cleanlab TLM API key: ")

os.environ["CLEANLAB_TLM_API_KEY"] = cleanlab_tlm_api_key

## Connect to Phoenix

In this example, we'll use Phoenix as our destination. You could instead add any other exporters you'd like in this approach.

If you need to set up an API key for Phoenix, you can do so [here](https://app.phoenix.arize.com/).

The code below will connect you to a Phoenix Cloud instance. You can also connect to [a self-hosted Phoenix instance](https://docs.arize.com/phoenix/deployment) if you'd prefer.

In [ ]:
# Add Phoenix API Key for tracing
if not (PHOENIX_API_KEY := os.getenv("PHOENIX_CLIENT_HEADERS")):
    PHOENIX_API_KEY = getpass("🔑 Enter your Phoenix API Key: ")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

Now that we have Phoenix configured, we can register that instance with OpenTelemetry, which will allow us to collect traces from our application here.

In [ ]:
from phoenix.otel import register

tracer_provider = register(project_name="evaluating_traces_TLM")

## Prepare trace dataset

For the sake of making this guide fully runnable, we'll briefly generate some traces and track them in Phoenix. Typically, you would have already captured traces in Phoenix and would skip to "Download trace dataset from Phoenix"

In [ ]:
from openinference.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()


# Function to generate an answer
def generate_answers(trivia_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a trivia master."},
            {"role": "user", "content": trivia_question},
        ],
    )
    answer = response.choices[0].message.content
    return answer


trivia_questions = [
    "What is the 3rd month of the year in alphabetical order?",
    "What is the capital of France?",
    "How many seconds are in 100 years?",
    "Alice, Bob, and Charlie went to a café. Alice paid twice as much as Bob, and Bob paid three times as much as Charlie. If the total bill was $72, how much did each person pay?",
    "When was the Declaration of Independence signed?",
]

# Generate answers
answers = []
for i in range(len(trivia_questions)):
    answer = generate_answers(trivia_questions[i])
    answers.append(answer)
    print(f"Question {i+1}: {trivia_questions[i]}")
    print(f"Answer {i+1}:\n{answer}\n")

print(f"Generated {len(answers)} answers and tracked them in Phoenix.")

## Download trace dataset from Phoenix

In [ ]:
import phoenix as px

spans_df = px.Client().get_spans_dataframe(project_name="evaluating_traces_TLM")
spans_df.head()

## Generate evaluations with TLM

Now that we have our trace dataset, we can generate evaluations for each trace using TLM. Ultimately, we want to end up with a trustworthiness score and explaination for each prompt, response pair in the traces.

In [ ]:
from cleanlab_tlm import TLM

tlm = TLM(options={"log": ["explanation"]})

We first need to extract the prompts and responses from the individual traces. `TLM.get_trustworthiness_score()` will take a list of prompts and responses and return trustworthiness scores and explanations.

**IMPORTANT:** It is essential to always include any system prompts, context, or other information that was originally provided to the LLM to generate the response. You should construct the prompt input to `get_trustworthiness_score()` in a way that is as similar as possible to the original prompt.

In [ ]:
# Create a new DataFrame with input and output columns
eval_df = spans_df[["context.span_id", "attributes.input.value", "attributes.output.value"]].copy()
eval_df.set_index("context.span_id", inplace=True)


# Combine system and user prompts from the traces
def get_prompt(input_value):
    if isinstance(input_value, str):
        input_value = json.loads(input_value)
    system_prompt = input_value["messages"][0]["content"]
    user_prompt = input_value["messages"][1]["content"]
    return system_prompt + "\n" + user_prompt


# Get the responses from the traces
def get_response(output_value):
    if isinstance(output_value, str):
        output_value = json.loads(output_value)
    return output_value["choices"][0]["message"]["content"]


# Create a list of prompts and associated responses
prompts = [get_prompt(input_value) for input_value in eval_df["attributes.input.value"]]
responses = [get_response(output_value) for output_value in eval_df["attributes.output.value"]]

eval_df["prompt"] = prompts
eval_df["response"] = responses

Now that we have all of the prompts and responses, we can evaluate each pair using TLM.

In [ ]:
# Evaluate each of the prompt, response pairs using TLM
evaluations = tlm.get_trustworthiness_score(prompts, responses)

# Extract the trustworthiness scores and explanations from the evaluations
trust_scores = [entry["trustworthiness_score"] for entry in evaluations]
explanations = [entry["log"]["explanation"] for entry in evaluations]

# Add the trust scores and explanations to the DataFrame
eval_df["score"] = trust_scores
eval_df["explanation"] = explanations

# Display the new DataFrame
eval_df.head()

We now have a DataFrame with added colums:
- `prompt`: the combined system and user prompt from the trace
- `response`: the LLM response from the trace
- `score`: the trustworthiness score from TLM
- `explanation`: the explanation from TLM

Let's sort our traces by the `score` column to quickly find untrustworthy LLM responses.

In [ ]:
sorted_df = eval_df.sort_values(by="score", ascending=True).head()
sorted_df

In [ ]:
# Let's look at the least trustworthy trace.
print("Prompt: ", sorted_df.iloc[0]["prompt"], "\n")
print("OpenAI Response: ", sorted_df.iloc[0]["response"], "\n")
print("TLM Trust Score: ", sorted_df.iloc[0]["score"], "\n")
print("TLM Explanation: ", sorted_df.iloc[0]["explanation"])

#### Awesome! TLM was able to identify multiple traces that contained incorrect answers from OpenAI.

Let's upload the `score` and `explanation` columns to Phoenix.

## Upload evaluations to Phoenix

Our evals_df has a column for the span_id and a column for the evaluation result. The span_id is what allows us to connect the evaluation to the correct trace in Phoenix. Phoenix will also automatically look for columns named "score" and "evaluation" to display in the UI. 

In [ ]:
eval_df["score"] = eval_df["score"].astype(float)
eval_df["explanation"] = eval_df["explanation"].astype(str)

In [ ]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(SpanEvaluations(eval_name="Trustworthiness", dataframe=eval_df))

You should now see evaluations in the Phoenix UI!

From here you can continue collecting and evaluating traces!